In [1]:
# Now -- decision tree, random forests, boostoing, etc SVM
# logistic regression
# x train, validate, test
# 
# do I need to regularlize?
#
# start with tech, then for all entries in my database 

In [2]:
# useful links
# https://github.com/jakevdp/sklearn_scipy2013/blob/master/rendered_notebooks/06.1_validation_and_testing.ipynb
# http://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html
# http://scikit-learn.org/stable/modules/generated/sklearn.metrics.precision_recall_fscore_support.html#sklearn.metrics.precision_recall_fscore_support
# http://scikit-learn.org/stable/modules/ensemble.html
# http://scikit-learn.org/stable/auto_examples/ensemble/plot_forest_importances.html

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math as math
from IPython.display import display
from IPython.core.pylabtools import getfigs
from sklearn import linear_model 
import seaborn as sns

In [4]:
%matplotlib qt

In [5]:
saveas_path = '/Users/jamie/insight data/figures/'

In [6]:
outcomes_df = pd.read_pickle('outcomes_df.save')

In [8]:
predictor_df_unprocessed = pd.read_pickle('predictor_df_unprocessed.save')

In [22]:
predictor_df_unprocessed.ix[8006,list(['Url'])].values

array(['https://www.kickstarter.com/projects/445200983/pawspa?ref=category'], dtype=object)

In [23]:
subset_predictor_df = pd.read_pickle('subset_predictor.save')
subset_predictor_df.head()

,Goal,Campaign_days,Title_n_words,Blurb_n_words,Month,Monday_day,Week_day,Hour,N_rewards,Min_reward,Median_reward,Max_reward,Reward_ship_days,Ships_intn,N_links,N_figs,Body_n_words,N_creator_backed
0,5000,30,2,17,8,12,6,20,8,10,175,5000,110,0,1,0,812,18
1,10000,30,8,21,6,30,5,14,10,5,125,5000,31,0,5,1,792,7
2,18000,27,5,14,2,25,1,8,19,5,500,10000,4,1,0,0,540,3
3,7000,56,5,17,12,6,1,17,8,10,87,1000,56,0,0,0,372,4
4,8150,30,8,16,7,22,1,11,11,4,42,815,41,-1,1,1,1405,8


In [24]:
subset_predictor_no_backers_df = subset_predictor_df.ix[:,0:-1]

In [25]:
subset_predictor_no_backers_df.head()

,Goal,Campaign_days,Title_n_words,Blurb_n_words,Month,Monday_day,Week_day,Hour,N_rewards,Min_reward,Median_reward,Max_reward,Reward_ship_days,Ships_intn,N_links,N_figs,Body_n_words
0,5000,30,2,17,8,12,6,20,8,10,175,5000,110,0,1,0,812
1,10000,30,8,21,6,30,5,14,10,5,125,5000,31,0,5,1,792
2,18000,27,5,14,2,25,1,8,19,5,500,10000,4,1,0,0,540
3,7000,56,5,17,12,6,1,17,8,10,87,1000,56,0,0,0,372
4,8150,30,8,16,7,22,1,11,11,4,42,815,41,-1,1,1,1405


In [26]:
outcomes_df.head()

,Outcome
0,1
1,1
2,1
3,1
4,1


In [27]:
n_samples = np.shape(outcomes_df)[0]
print n_samples

133071


In [28]:
# now use training (60%), validatation (20%), and test (20%) sets 
i_order = np.random.permutation(n_samples)
train_max_i = int(round(n_samples*.6))
validate_max_i = int(round(n_samples*.8))

train_is = i_order[:train_max_i]
validate_is = i_order[train_max_i:validate_max_i]
test_is = i_order[validate_max_i:]

In [29]:
X = subset_predictor_df
X_subset = subset_predictor_no_backers_df
y = outcomes_df['Outcome'].values

In [30]:
# now try a random forests model
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators=200, max_depth=8)
clf.fit(X.ix[train_is,:],y[train_is])
y_1 = clf.predict(X.ix[test_is,:])

y_test = y[test_is]

In [31]:
accuracy = float(sum(y_test == y_1))/len(test_is)
print accuracy

0.730855940482


In [41]:
from sklearn.externals import joblib
app_path = '/Users/jamie/app/pickled_clf/'
joblib.dump(clf, app_path+'random_forests_clf.pkl') 

['/Users/jamie/app/pickled_clf/random_forests_clf.pkl',
 '/Users/jamie/app/pickled_clf/random_forests_clf.pkl_01.npy',
 '/Users/jamie/app/pickled_clf/random_forests_clf.pkl_02.npy',
 '/Users/jamie/app/pickled_clf/random_forests_clf.pkl_03.npy',
 '/Users/jamie/app/pickled_clf/random_forests_clf.pkl_04.npy',
 '/Users/jamie/app/pickled_clf/random_forests_clf.pkl_05.npy',
 '/Users/jamie/app/pickled_clf/random_forests_clf.pkl_06.npy',
 '/Users/jamie/app/pickled_clf/random_forests_clf.pkl_07.npy',
 '/Users/jamie/app/pickled_clf/random_forests_clf.pkl_08.npy',
 '/Users/jamie/app/pickled_clf/random_forests_clf.pkl_09.npy',
 '/Users/jamie/app/pickled_clf/random_forests_clf.pkl_10.npy',
 '/Users/jamie/app/pickled_clf/random_forests_clf.pkl_11.npy',
 '/Users/jamie/app/pickled_clf/random_forests_clf.pkl_12.npy',
 '/Users/jamie/app/pickled_clf/random_forests_clf.pkl_13.npy',
 '/Users/jamie/app/pickled_clf/random_forests_clf.pkl_14.npy',
 '/Users/jamie/app/pickled_clf/random_forests_clf.pkl_15.npy',

In [44]:
#Later you can load back the pickled model (possibly in another Python process) with:
clf_p = joblib.load(app_path+'random_forests_clf.pkl') 
X = pd.read_pickle('subset_predictor.save')

random_i = np.random.randint(0,n_campaigns)
predicted_outcome = clf.predict(X.ix[random_i,:])


if predicted_outcome:
    return success
else:
    return failure;

[1]


In [49]:
n_campaigns = np.shape(X)[0]
print n_campaigns

133071


In [51]:
print 

1262
